In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Load and preprocess data
text = "Today is a good day. We are in the lab"
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in text.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences and create predictors and label
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
label = tf.keras.utils.to_categorical(label, num_classes=total_words)

# Define the model
model = Sequential([
    Embedding(total_words, 100, input_length=max_sequence_len-1),
    LSTM(150),
    Dense(total_words, activation='softmax')
])

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(predictors, label, epochs=100, verbose=1)




Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 2.3964 - accuracy: 0.1250
Epoch 2/100
1/1 [==============================] - 0s 18ms/step - loss: 2.3861 - accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 17ms/step - loss: 2.3757 - accuracy: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 18ms/step - loss: 2.3650 - accuracy: 0.6250
Epoch 5/100
1/1 [==============================] - 0s 18ms/step - loss: 2.3538 - accuracy: 0.6250
Epoch 6/100
1/1 [==============================] - 0s 16ms/step - loss: 2.3421 - accuracy: 0.6250
Epoch 7/100
1/1 [==============================] - 0s 18ms/step - loss: 2.3296 - accuracy: 0.6250
Epoch 8/100
1/1 [==============================] - 0s 19ms/step - loss: 2.3162 - accuracy: 0.6250
Epoch 9/100
1/1 [==============================] - 0s 16ms/step - loss: 2.3018 - accuracy: 0.7500
Epoch 10/100
1/1 [==============================] - 0s 17ms/step - loss: 2.2860 - accuracy: 0.7500
Epoch 11/100
1/1 [===

In [ ]:
import numpy as np
# Text generation
def generate_text(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word = tokenizer.index_word[np.argmax(predicted)]
        seed_text += " " + predicted_word
    return seed_text

print(generate_text("Your seed text", 10))

Your seed text is is a good day day day day lab lab


In [ ]:
link_to_dataset = "https://www.kaggle.com/datasets/narendrageek/mental-health-faq-for-chatbot"

install dependencies

In [ ]:
%pip install -r '/content/requirements.txt'

ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/content/requirements.txt'


In [ ]:
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import pandas as pd
import nltk
import numpy as np
import re

from nltk.stem import wordnet                                  # to perform lemmitization
from sklearn.feature_extraction.text import CountVectorizer    # to perform bow
from sklearn.feature_extraction.text import TfidfVectorizer    # to perform tfidf
from nltk import pos_tag                                       # for parts of speech
from sklearn.metrics import pairwise_distances                 # to perfrom cosine similarity
from nltk import word_tokenize                                 # to create tokens
from nltk.corpus import stopwords                              # for stop words

In [ ]:
!unzip /content/mental-health-faq.zip -d /content/

Archive:  /content/mental-health-faq.zip
  inflating: /content/Mental_Health_FAQ.csv  


In [ ]:
path = '/content/Mental_Health_FAQ.csv'

In [ ]:
df = pd.read_csv(path)
df.head()

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['Questions']], df[['Answers']], random_state=42)

In [ ]:
X_train.shape

(73, 1)

In [ ]:
X_test.shape

(25, 1)

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
s = 'I feel sad a lot. Is that normal?'

In [ ]:
lemma = wordnet.WordNetLemmatizer()
lemma.lemmatize("absorbed", pos = 'v')

'absorb'

In [ ]:
pos_tag(nltk.word_tokenize(s),tagset = None)

In [ ]:
stop = stopwords.words('english')
stop

In [ ]:
def text_normalization(text):
    text = str(text).lower()

    spl_char_text = re.sub(r'[^ a-z]','',text)

    tokens = nltk.word_tokenize(spl_char_text)

    lema = wordnet.WordNetLemmatizer()

    tags_list = pos_tag(tokens,tagset=None)

    lema_words = []

    for token,pos_token in tags_list:
        if pos_token.startswith('V'):               # verb
            pos_val = 'v'
        elif pos_token.startswith('J'):             # adjective
            pos_val = 'a'
        elif pos_token.startswith('R'):             # adverb
            pos_val = 'r'
        else:
            pos_val = 'n'                           # noun
        lema_token = lema.lemmatize(token,pos_val)

        if lema_token in stop:
          lema_words.append(lema_token)             # appending the lemmatized token into a list

    return " ".join(lema_words)

In [ ]:
text_normalization("There is something i want to tell you")

In [ ]:
df['lemmatized_text'] = df['Questions'].apply(text_normalization)

In [ ]:
df.head()

In [ ]:
cv = CountVectorizer()
X = cv.fit_transform(df['lemmatized_text']).toarray()

In [ ]:
features = cv.get_feature_names_out()
df_bow = pd.DataFrame(X, columns = features)
df_bow.head()

In [ ]:
Question = 'What treatment options are available?'
Question_lemma = text_normalization(Question)
Question_bow = cv.transform([Question_lemma]).toarray()

In [ ]:
Question_lemma

In [ ]:
Question_bow

In [ ]:
cosine_value = 1- pairwise_distances(df_bow, Question_bow, metric = 'cosine' )
(cosine_value)

In [ ]:
df['similarity_bow'] = cosine_value

In [ ]:
df.head()

In [ ]:
df.sort_values(by = 'similarity_bow', ascending=False).head()  # sorting the values

In [ ]:
df.sort_values(by = 'similarity_bow', ascending=False).tail()

In [ ]:
threshold = 0.1 #considering the value of smiliarity to be greater than 0.1
df[df['similarity_bow'] > threshold]

In [ ]:
Question1 = 'What treatment options are available'

In [ ]:
tfidf = TfidfVectorizer()
x_tfidf = tfidf.fit_transform(df['lemmatized_text']).toarray()

In [ ]:
Question_lemma1 = text_normalization(Question1)
Question_tfidf = tfidf.transform([Question_lemma1]).toarray()

In [ ]:
Question_lemma1

In [ ]:
Question_tfidf

In [ ]:
df_tfidf = pd.DataFrame(x_tfidf,columns = tfidf.get_feature_names_out())
df_tfidf.head()

In [ ]:
cos = 1-pairwise_distances(df_tfidf,Question_tfidf,metric='cosine')

In [ ]:
df['similarity_tfidf'] = cos                                                    # creating a new column
df_simi_tfidf = pd.DataFrame(df, columns=['Answers','similarity_tfidf'])        # taking similarity value of responses for the question we took
df_simi_tfidf

In [ ]:
df_simi_tfidf.sort_values(by='similarity_tfidf', ascending=False).head(10)

In [ ]:
threshold = 0.1
df_simi_tfidf[df_simi_tfidf['similarity_tfidf'] > threshold]

In [ ]:
df['Answers'].loc[6]

In [ ]:
def chat_bow(text): #chatbot based on Bag-Of-Words Model
    lemma = text_normalization(text)
    bow = cv.transform([lemma]).toarray()
    cosine_value = 1- pairwise_distances(df_bow,bow, metric = 'cosine' )
    index_value = cosine_value.argmax()

    return df['Answers'].loc[index_value]

def chat_tfidf(text): #chatbot based on TF-IDF model
    lemma = text_normalization(text)
    tf = tfidf.transform([lemma]).toarray()
    cos = 1-pairwise_distances(df_tfidf,tf,metric='cosine')
    index_value = cos.argmax()
    return df['Answers'].loc[index_value]

In [ ]:
chat_bow('can you prevent mental health problems')

'We can all suffer from mental health challenges, but developing our wellbeing, resilience, and seeking help early can help prevent challenges becoming serious.'

In [ ]:
chat_bow('what is mental health')

'Just as there are different types of medications for physical illness, different treatment options are available for individuals with mental illness. Treatment works differently for different people. It is important to find what works best for you or your child.'

In [ ]:
chat_tfidf('how do i see a counsellor')

'If your beliefs , thoughts , feelings or behaviours have a significant impact on your ability to function in what might be considered a normal or ordinary way, it would be important to seek help.'

In [ ]:
print(chat_tfidf('how to find a support group'))

Distraction is a very valid tool to help you cope when everything feels overwhelming or when you feel lonely or isolated. 
 If you don't have a lot of energy or focus right now, try low-effort distractions like watching TV, browsing Youtube, listening to a podcast or audiobook, playing a game on your phone, reading an easy book or magazine, or working on a simple art project. 
 If you have more energy and focus, give yourself a to-do list every day: you can clean and take care of projects around your home, work on hobbies, connect with family or friends, read a new book and catch up on your favourite TV shows. You can find interesting opportunities to take online courses from universities all over the world through MOOCs and other online learning platforms, you can learn a new language online or through apps, and you can learn new hobbies and activities. As more people have to practice social distancing or self-isolation, people are finding creative ways to bring the world into their h

In [ ]:
chat_bow("what should i do if i am worried about a friend?")

'This may depend on your relationship with them. Gently encouraging someone to seek appropriate support would be helpful to start with.'

In [ ]:
chat_bow("what are warning signs of depression mental health?")

'There are many types of mental health professionals. Finding the right one for you may require some research.'